In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
'''
from keras.models import Sequential
from keras.layers import LSTM, Dense

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dropout, Dense, Flatten
from tensorflow.keras.layers import BatchNormalization
'''

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from sklearn.metrics import f1_score
from sklearn.metrics import precision_recall_curve

: 

In [2]:
#df = pd.read_csv('data/train.csv')
df_labels = pd.read_csv('data/raw/train_labels.csv')

In [3]:
def clean_df(train):
    # fqid values that occur less than 100 times in train set are set to 'rare', nan values are set to 'missing'
    rare_values = train['fqid'].value_counts()[train['fqid'].value_counts() < 1000].index

    train['fqid'] = train['fqid'].apply(lambda x: 'rare' if x in rare_values else x)
    train['fqid'] = train['fqid'].fillna('missing')



    # text_fqid values that occur less than 100 times in train set are set to 'rare', nan values are set to 'missing'
    rare_values = train['text_fqid'].value_counts()[train['text_fqid'].value_counts() < 1000].index

    train['text_fqid'] = train['text_fqid'].apply(lambda x: 'rare' if x in rare_values else x)
    train['text_fqid'] = train['text_fqid'].fillna('missing')


    # text values that occur less than 1000 times in train set are set to 'rare', nan values are set to 'none'
    rare_values = train['text'].value_counts()[train['text'].value_counts() < 1000].index

    train['text'] = train['text'].apply(lambda x: 'rare' if x in rare_values else x)
    train['text'] = train['text'].fillna('none')


    # fill coor cols with 0
    train['room_coor_x'] = train['room_coor_x'].fillna(0)
    train['room_coor_y'] = train['room_coor_y'].fillna(0)

    train['screen_coor_x'] = train['screen_coor_x'].fillna(0)
    train['screen_coor_y'] = train['screen_coor_y'].fillna(0)


    # fill page cols with -1
    train['page'] = train['page'].fillna(-1)


    # fill hover_duration with -1
    train['hover_duration'] = train['hover_duration'].fillna(-1)


    train["weekday"] = train["session_id"].apply(lambda x: int(str(x)[4:6])).astype(np.uint8)
    train["hour"] = train["session_id"].apply(lambda x: int(str(x)[6:8])).astype(np.uint8)

    train.drop(['hq', 'music'], axis=1, inplace=True)

    # event index starts from 0 for each session
    train['event_index'] = train.groupby('session_id').cumcount()
    # drop index column
    train = train.drop(['index'], axis=1)

    # Make each user's elapsed time into difference from previous row
    train['elapsed_time_diff'] = train.groupby('session_id')['elapsed_time'].diff().fillna(0)

    # fill nan or inf with 0
    train = train.replace([np.inf, -np.inf], np.nan)
    train = train.fillna(0)

    # get columns that are object type
    obj_cols = train.select_dtypes(include=['object']).columns

    # if category in obj_cols has less than 1000 counts, replace category with 'other'
    for col in obj_cols:
        counts = train[col].value_counts()
        rare_cats = counts[counts < 1000].index.tolist()
        train[col] = train[col].apply(lambda x: 'other' if x in rare_cats else x)

    train = pd.get_dummies(train, columns=obj_cols)
    return train

In [4]:
#df = clean_df(df)

In [5]:
#df.to_pickle('train_checkpoint.pkl')

In [6]:
df = pd.read_pickle('train_checkpoint.pkl')



In [7]:
def create_sequence_X(session_id):
    sequence = df[df['session_id'] == session_id]

    X = sequence.drop(['session_id'], axis=1).values

    # if sequence is not 220, pad with zeros
    if X.shape[0] < 220:
        X = np.pad(X, ((0, 220-X.shape[0]), (0,0)), 'constant')

    if X.shape[0] > 220:
        X = X[:220]
    del sequence
    return X




Xs = []

for session_id in df['session_id'].unique()[0:100]:
    X = create_sequence_X(session_id)
    if X.shape[0] > 1: # filter out sessions with only one event
        Xs.append(X)

Xs = np.array(Xs)


In [9]:
def create_sequence_X(group):
    X = group.drop(['session_id'], axis=1).values

    # if sequence is not 220, pad with zeros
    if X.shape[0] < 220:
        X = np.pad(X, ((0, 220-X.shape[0]), (0,0)), 'constant')

    if X.shape[0] > 220:
        X = X[:220]

    return X

Xs = df.groupby('session_id').apply(create_sequence_X)
Xs = np.concatenate(Xs[Xs[:, :, 0] != 0], axis=0)


: 

: 

In [5]:
!pip uninstall numpy -y
!pip install numpy==1.18.0


Found existing installation: numpy 1.22.0
Uninstalling numpy-1.22.0:
  Successfully uninstalled numpy-1.22.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 19.6 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for numpy (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [391 lines of output]
      Running from numpy source directory.
      Cythonizing sources
      numpy/random/_bounded_integers.pxd.in has not changed
      numpy/random/_philox.pyx has not changed
      numpy/random/_bit_generator.pyx has not changed
      numpy/random/_bounded_integers.pyx.in has not changed
      numpy/random/_sfc64.pyx has not changed
      numpy/random/_mt19937.pyx has not changed
      Processing numpy/random/_bounded_integers.pyx
      /private/var/folders/xh/rszsbt095y15wwjg15z_51lh0000gn/T/pip-install